In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

This notebook is used to test the conversion of non-deterministic <span style="font-variant:small-caps;">Fsm</span>s into deterministic <span style="font-variant:small-caps;">Fsm</span>s.

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, Value } from "recursive-set";

In [ ]:
const viz = await instance();

In order to represent the <span style="font-variant:small-caps;">Fsm</span>s graphically, we use the notebook `FSM-2-Dot.ipynb`.  This notebook uses the package `graphviz`.

In [ ]:
import { NFA, DFA, nfa2dfa, TransRel, TransitionKey, State, Char, key } from "./01-NFA-2-DFA";
import { nfa2dot, dfa2dot, nfa2string, dfa2string } from "./FSM-2-Dot";

In [ ]:
const States: RecursiveSet<State> = RecursiveSet.fromArray(
  Array.from({ length: 8 }, (_, i) => `q${i}`)
);
States;

In [ ]:
const Sigma: RecursiveSet<Char> = new RecursiveSet("a", "b");

In [ ]:
const delta: TransRel = new Map<TransitionKey, RecursiveSet<State>>([
  [key("q0", "ε"), new RecursiveSet("q1", "q2")],
  [key("q1", "b"), new RecursiveSet("q3")],
  [key("q2", "a"), new RecursiveSet("q4")],
  [key("q3", "a"), new RecursiveSet("q5")],
  [key("q4", "b"), new RecursiveSet("q6")],
  [key("q5", "ε"), new RecursiveSet("q7")],
  [key("q6", "ε"), new RecursiveSet("q7")],
  [key("q7", "ε"), new RecursiveSet("q0")],
]);
delta;

The non-deterministic <span style="font-variant:small-caps;">Fsm</span> defined below is taken from the lecture notes.

In [ ]:
const nfa44: NFA = {
  Q: States,
  Sigma: Sigma,
  delta: delta,
  q0: "q0",
  A: new RecursiveSet("q7"),
};

The function `nfa2dot`can be used to render this <span style="font-variant:small-caps;">Fsm</span>.

In [ ]:
const dotNFA: string = nfa2dot(nfa44);

In [ ]:
display.html(viz.renderString(dotNFA, { format: "svg" }));

This recognizes the same language as the language described by
$$ (a \cdot b + b \cdot a) \cdot (a \cdot b + b \cdot a)^* $$

Let us convert it into a deterministic Fsm:

In [ ]:
const dfa44: DFA = nfa2dfa(nfa44);

The function `dfa2dot`can be used to render this <span style="font-variant:small-caps;">Fsm</span>.

In [ ]:
const { dot: dotDFA, statesToNames }: { 
    dot: string, 
    statesToNames: Map<RecursiveSet<State>, string> 
} = dfa2dot(dfa44);

In [ ]:
display.html(viz.renderString(dotDFA, { format: "svg" }));

In order to inspect the states of this deterministic <span style="font-variant:small-caps;">Fsm</span> we print the dictionary  `states2Names`.

In [ ]:
statesToNames;

We can also print the <span style="font-variant:small-caps;">Fsm</span>.

In [ ]:
dfa2string(dfa44);